<a href="https://colab.research.google.com/github/mariamlaagrini/scrape-products/blob/main/scrape_products.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import json
import gspread
import pandas as pd
from bs4 import BeautifulSoup
from oauth2client.service_account import ServiceAccountCredentials

# 1️⃣ Read and Parse the HTML File
with open("NATRUE.html", "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")

# 2️⃣ Extract Product Details
products = []

for item in soup.find_all("article", class_="product-list__item"):
    name = item.find("div", class_="product-list__item__name")
    brand = item.find("div", class_="product-list__item__brand")
    image = item.find("div", class_="product-list__item__image")

    product_data = {
        "Name": name.get_text(strip=True) if name else "N/A",
        "Brand": brand.get_text(strip=True) if brand else "N/A",
        "Image_URL": image.find("img")["src"] if image and image.find("img") else "N/A"
    }
    products.append(product_data)

# 3️⃣ Save Data as JSON
with open("products.json", "w", encoding="utf-8") as json_file:
    json.dump(products, json_file, indent=4, ensure_ascii=False)

print("✅ Data saved to products.json")

# 4️⃣ Upload Data to Google Sheets
def upload_to_google_sheets(data, credentials_file, sheet_name):
    # Define API scope
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

    # Authenticate with Google Sheets
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_file, scope)
    client = gspread.authorize(creds)

    # Open the spreadsheet (Create if not exists)
    try:
        sheet = client.open(sheet_name).sheet1
    except gspread.exceptions.SpreadsheetNotFound:
        print(f"⚠️ Spreadsheet '{sheet_name}' not found. Please create it first.")
        return

    # Convert data to DataFrame
    df = pd.DataFrame(data)

    # Clear existing data
    sheet.clear()

    # Upload data (including headers)
    sheet.update([df.columns.values.tolist()] + df.values.tolist())
    print(f"✅ Data uploaded successfully to '{sheet_name}' on Google Sheets.")

# 5️⃣ Provide Credentials and Sheet Name
CREDENTIALS_FILE = "credentials.json"  # Replace with your JSON file path
SHEET_NAME = "products"

upload_to_google_sheets(products, CREDENTIALS_FILE, SHEET_NAME)


✅ Data saved to products.json
✅ Data uploaded successfully to 'products' on Google Sheets.


In [10]:
with open("NATRUE brand.html", "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")


brands = []

for item in soup.find_all("article", class_="brand-list__item"):
    name = item.find("div", class_="brand-list__item__name")
    image = item.find("div", class_="brand-list__item__image")

    brand_data = {
        "Name": name.get_text(strip=True) if name else "N/A",
        "Image_URL": image.find("img")["src"] if image and image.find("img") else "N/A"
    }
    brands.append(brand_data)

with open("brand.json", "w", encoding="utf-8") as json_file:
    json.dump(brands, json_file, indent=4, ensure_ascii=False)

print("✅ Data saved to brand.json")

def upload_to_google_sheets(data, credentials_file, sheet_name):
    # Define API scope
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

    # Authenticate with Google Sheets
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_file, scope)
    client = gspread.authorize(creds)

    # Open the spreadsheet (Create if not exists)
    try:
        sheet = client.open(sheet_name).sheet1
    except gspread.exceptions.SpreadsheetNotFound:
        print(f"⚠️ Spreadsheet '{sheet_name}' not found. Please create it first.")
        return

    # Convert data to DataFrame
    df = pd.DataFrame(data)

    # Clear existing data
    sheet.clear()

    # Upload data (including headers)
    sheet.update([df.columns.values.tolist()] + df.values.tolist())
    print(f"✅ Data uploaded successfully to '{sheet_name}' on Google Sheets.")

# 5️⃣ Provide Credentials and Sheet Name
CREDENTIALS_FILE = "credentials.json"  # Replace with your JSON file path
SHEET_NAME = "brand"

upload_to_google_sheets(brands, CREDENTIALS_FILE, SHEET_NAME)

✅ Data saved to brand.json
✅ Data uploaded successfully to 'brand' on Google Sheets.


In [7]:
!pip install pandas gspread oauth2client

In [9]:
import pandas as pd
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Step 1: Load the Excel file
excel_file = "raw_materials.xlsx"  # Change to your actual file path
df = pd.read_excel(excel_file)

# Step 2: Convert to JSON and save locally
json_file = "raw_materials.json"
data_json = df.to_dict(orient="records")

with open(json_file, "w", encoding="utf-8") as f:
    json.dump(data_json, f, indent=4, ensure_ascii=False)

print(f"JSON saved successfully as {json_file}")

# Step 3: Upload to Google Sheets
# Google Sheets Authentication
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
client = gspread.authorize(creds)

sheet = client.open("raw materials").sheet1

# Upload data to Google Sheets
sheet.clear()  # Clear existing data
data_to_upload = df.values.tolist()
for row_index, row in enumerate(data_to_upload):
    for col_index, value in enumerate(row):
        if isinstance(value, float) and (float('inf') == value or float('-inf') == value or pd.isna(value)):
            data_to_upload[row_index][col_index] = str(value)  # Replace inf, -inf, and NaN with strings

sheet.append_rows(data_to_upload)
print("Data successfully uploaded to Google Sheets!")


JSON saved successfully as raw_materials.json
Data successfully uploaded to Google Sheets!
